# Querying Chinook database to answer Business questions

Chinook.db schema is provided [here](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

Below we connect to the database file on dataquest

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ("table","view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist name | Genre   |
|-------------|---------|
| Regal       | Hip-Hop |
| Red Tone    | Punk    |
|Meteor and the Girls|Pop|
|Slim Jim Bites|Blues|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [3]:
%%sql
WITH us_customer AS
       ( SELECT t.genre_id, il.*
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    INNER JOIN track t ON t.track_id = il.track_id
       WHERE c.country = "USA" )
    
SELECT g.name Genre_name, 
    COUNT(usc.quantity) tracks_sold,
    CAST ( COUNT(usc.quantity) AS FLOAT )/(SELECT COUNT(*) FROM us_customer) Percentage_sol
FROM genre g
INNER JOIN us_customer usc ON g.genre_id = usc.genre_id
GROUP BY 1
ORDER BY 2 DESC

Done.


Genre_name,tracks_sold,Percentage_sol
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


From on the above results the three albums that should be selected for advertisement by Chinook based on genre are:
* Red Tone (Punk)
* Slim Jim Bites (Blues)
* Meteor and the Girls (Pop)

We should also be on the look out for Artistes in the Rock genre, since rock albums are responsible for 53% of sales.

## Employee sales performance

In [4]:
%%sql
SELECT e.first_name||' '||e.last_name employee,
        e.hire_date date_hired,
        SUM(i.total) Total_sales
FROM employee e
INNER JOIN customer c ON e.employee_id = c.support_rep_id
INNER JOIN invoice i  ON c.customer_id = i.customer_id
GROUP BY 1;

Done.


employee,date_hired,Total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


We can see that Jane Peacock is the employee with the highest sales but this can be attributed the fact that she was the first to be hired.

## Sales by country

We want to aggregate data on purchases by countries, with some guidelines;
* where a country has only one customer, collect them into an "Other" group.
* The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

In [46]:
%%sql
WITH country_summary AS
    ( SELECT CASE WHEN 
                     (SELECT count(*)
                      FROM customer
                      WHERE country = c.country
                        ) = 1 THEN "Other"
               ELSE c.country END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     JOIN invoice i ON i.invoice_id = il.invoice_id
     JOIN customer c ON c.customer_id = i.customer_id
    )
    
SELECT  country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) average_sales_customer,
        SUM(unit_price) / count(distinct invoice_id) average_order_price,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_summary
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC

Done.


country,customers,total_sales,average_sales_customer,average_order_price,sort
USA,13,1040.490000000008,80.03769230769292,7.942671755725252,0
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309,0
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647,0
France,5,389.0700000000021,77.81400000000042,7.781400000000042,0
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186,0
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034,0
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457,0
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835,0
India,2,183.1500000000002,91.5750000000001,8.72142857142858,0
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486,1
